In [1]:
import intake
import easygems.healpix as egh
import numpy as np
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt
import matplotlib as mpl
import cartopy.crs as ccrs
from datetime import datetime
import glob
import logging
import os
import pathlib
import dask
dask.config.set(**{'array.slicing.split_large_chunks': True})
import global3d_track as g3d
src = g3d.scripts.src

outdir = pathlib.Path(f'/work/bb1153/b382635/plots/tracked_results_2025/dataset_paper/results_data/acp_submission/')
os.makedirs(outdir, exist_ok=True)

In [2]:
# choose times to plot

t1 = pd.to_datetime("2021-07-01T16:00")
t2 = pd.to_datetime("2021-07-02T04:00")

In [3]:
# load icon data
logging.info(f"{datetime.now()} loading model data")
cat = intake.open_catalog("https://data.nextgems-h2020.eu/catalog.yaml")
globaldata = cat.ICON.ngc4008a(time="PT15M", zoom=9).to_dask().sel(time=slice('20210701','20210708'))

INFO:root:2026-01-29 18:29:10.108941 loading model data


In [4]:
# to lat lon
d = globaldata
d['time'] = d.time - pd.Timedelta(hours=4) # to AMT time
data = src.utils.regrid.Regrid('amazon').perform(d.sel(time=[t1,t2]), zoom=9, resolution=0.1)
data = src.utils.data_tools.preprocess_for_tobac(data).chunk({'time': 1, 'lat': 100, 'lon': 100})

INFO:root:Region to regrid: (-83, -43, -15, 15)


In [5]:
# compute IWP
iwp = src.statistics.density.calculate_IWP(data).compute()
lwp = src.statistics.density.calculate_xWP(data, 'clw').compute()

In [6]:
# load all tracking results
ex_tracks = '/work/bb1153/b382635/data/final_tracks/updraft_ice_only/amazon/20210701/20210701T0000_20210702T0000_system_tracks_linked_proc.nc'
mask_coords = xr.open_mfdataset(ex_tracks).drop_dims('time').coords
files = sorted(glob.glob('/work/bb1153/b382635/data/track_statistics/updraft_ice_only/amazon/first_bucket/*.nc'))
li = []
for f in files:
    li.append(xr.Dataset(coords=mask_coords).merge(xr.open_mfdataset(f))[['core_area', 'att','cloud_area']])
ds = xr.concat(li, dim='time')
ds = ds.where(ds != -999.99)

# subset times
ds['time'] = ds.time - pd.Timedelta(hours=4) # to AMT time
ds['time'].attrs = {'time_zone':'AMT'}
ds_small = ds.sel(time=[t1,t2])

In [7]:
ds_small['n_cores'] = ds_small.core_area.max(('time')).count('core')
ds_small = ds_small.drop_vars('core_area')
ds_small = ds_small.drop_dims('core')

In [8]:
ds_small['iwp'] = iwp
ds_small['lwp'] = lwp
ds_small.to_netcdf(outdir / 'example_scenes.nc')

In [10]:
# prep data for timeseries (valid data only)

# - validity
df = pd.read_csv(outdir / 'system_validity.csv', index_col='system_id')
invalid_convection = (df.n_cores_above_freezing / df.n_cores) == 1
invalid_rule = df.hits_boundary | invalid_convection
valid = df.index[~invalid_rule]
ds = ds.sel(system=valid)

# - results
area = ds.cloud_area.mean('system').compute() / 1e6 # convert units [m2] -> [km2]
total_systems = (ds.cloud_area>0).sum('system').compute()
n_cores = (ds.core_area>0).sum('core').compute()
n_cores_time = n_cores.sum('system') / total_systems # avg per system

In [11]:
df_series = area.to_dataframe(name='mean_dcc_area')
df_series['n_cores_per_dcc'] = n_cores_time.values
df_series['n_dccs'] = total_systems.values
df_series.to_csv(outdir / 'timeseries.csv')